In [1]:
import solution_file_processing as sfp
import os
import numpy as np
import pandas as pd
from dask import dataframe as dd

from solution_file_processing.utils.utils import catch_errors
from solution_file_processing.utils.write_excel import write_xlsx_column, write_xlsx_stack, STACK_PALETTE, IEA_PALETTE_16, IEA_PALETTE_PLUS, EXTENDED_PALETTE
from solution_file_processing.constants import VRE_TECHS, PRETTY_MODEL_NAMES
from solution_file_processing.timeseries import create_output_11 as create_ts_output_11
from solution_file_processing.timeseries import create_output_4 as create_timeseries_output_4
from solution_file_processing import log
from solution_file_processing.plots import _get_plot_1_variables

# Initialize confi||g with toml file
c = sfp.SolutionFilesConfig('config_files/china/CHN.toml')

# c = sfp.SolutionFilesConfig('config_files/thailand/THA.toml')
ix = pd.IndexSlice  

from solution_file_processing.plots import create_plot_1a, create_plot_2b_ref_plots, create_plot_2, create_plot_10_ts_by_model
from solution_file_processing.plots import create_plot_6_ldc, create_plot_7_co2_savings, create_plot_8_services, create_plot_9_av_cap

[17:28:02 00:00] I:119:__init__ - Logging to C:\Users\hart_c\showcase\solution-file-processing\logs\20240827_172802-CHN.log.
[17:28:04 00:01] I:156:__init__ - Initialized SolutionFilesConfig for config_files/china/CHN.toml.


In [3]:
c.v.net_load_ts

c.v.net_load_orig_ts

[17:31:24 03:05] I:47:_drive_cache_wrapper - Loading from variables cache: ramp_by_gen_subtech_ts.parquet (pd.DataFrame).


CapacityCategory                            Bioenergy  Coal_IGCC  \
model                timestamp                                     
2030_APS-FDp-ASb-SM  2030-01-01 00:00:00     0.000000        0.0   
                     2030-01-01 01:00:00   716.680000        0.0   
                     2030-01-01 02:00:00     0.000000        0.0   
                     2030-01-01 03:00:00  -729.800000        0.0   
                     2030-01-01 04:00:00     0.000000        0.0   
...                                               ...        ...   
2030_APS-FDp-SM_nBAS 2030-12-31 19:00:00 -1110.504631        0.0   
                     2030-12-31 20:00:00     0.000000        0.0   
                     2030-12-31 21:00:00   647.676881        0.0   
                     2030-12-31 22:00:00    70.213119        0.0   
                     2030-12-31 23:00:00   697.914020        0.0   

CapacityCategory                          Coal_Subcritical  \
model                timestamp                               
2030_APS-FDp-ASb-SM  2030-01-01 00:00:00              0.00   
                     2030-01-01 01:00:00            -77.72   
                     2030-01-01 02:00:00              0.00   
                     2030-01-01 03:00:00            -81.46   
                     2030-01-01 04:00:00              0.00   
...                                                    ...   
2030_APS-FDp-SM_nBAS 2030-12-31 19:00:00              0.00   
                     2030-12-31 20:00:00              0.00   
                     2030-12-31 21:00:00              0.00   
                     2030-12-31 22:00:00              0.00   
                     2030-12-31 23:00:00              0.00   

CapacityCategory                          Coal_Supercritical    Gas_CCGT  \
model                timestamp                                             
2030_APS-FDp-ASb-SM  2030-01-01 00:00:00            0.000000     0.00000   
                     2030-01-01 01:00:00         -380.340000     0.00000   
                     2030-01-01 02:00:00            0.000000 -1056.40000   
                     2030-01-01 03:00:00            0.000000     0.00000   
                     2030-01-01 04:00:00            0.000000   -19.06000   
...                                                      ...         ...   
2030_APS-FDp-SM_nBAS 2030-12-31 19:00:00            0.000000     0.00000   
                     2030-12-31 20:00:00           21.402961     0.00000   
                     2030-12-31 21:00:00            0.000000     0.00000   
                     2030-12-31 22:00:00         1444.878693  2918.50303   
                     2030-12-31 23:00:00            0.000000     0.00000   

CapacityCategory                          Gas_GT  Gas_steam        Hydro  \
model                timestamp                                             
2030_APS-FDp-ASb-SM  2030-01-01 00:00:00     0.0        0.0     0.000000   
                     2030-01-01 01:00:00     0.0        0.0  -845.021567   
                     2030-01-01 02:00:00     0.0        0.0   622.960439   
                     2030-01-01 03:00:00     0.0        0.0   242.292458   
                     2030-01-01 04:00:00     0.0        0.0   216.702519   
...                                          ...        ...          ...   
2030_APS-FDp-SM_nBAS 2030-12-31 19:00:00     0.0        0.0 -5258.683339   
                     2030-12-31 20:00:00     0.0        0.0  -207.192187   
                     2030-12-31 21:00:00     0.0        0.0   650.363233   
                     2030-12-31 22:00:00   167.2        0.0  7300.901530   
                     2030-12-31 23:00:00     0.0        0.0  1088.604482   

CapacityCategory                          ConcentratedSolarPower  SolarPV  \
model                timestamp                                              
2030_APS-FDp-ASb-SM  2030-01-01 00:00:00                0.000000      0.0   
                     2030-01-01 01:00:00              267.577664      0.0   
                     2